In [ ]:
import datasets
import plotly.io as pio
import datetime
from pyHR import extraction
from utils import speed_acc_sig,sum_speed_acc_sig
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [ ]:
today = str(datetime.date.today())
pio.renderers.default = 'browser'

In [ ]:
dataset_name = 'ecg_fitness'                              # 数据集名称
video_DIR = r"D:\Data_Set\pyVHR\ECG_FITNESS Signal"         # 视频数据目录的路径
BVP_DIR = r"D:\Data_Set\pyVHR\ECG_FITNESS Signal"             # 相应的BVP数据目录的路径

w_size = 20                # 每次处理视频的滑窗长度 ，w_size=视频时长，则timesGT的第一个值对应的bpmGT中的心率就是对应整个视频时长的平均心率
stride = 1                # 相邻滑窗在时间上的重叠长度
seconds = 0               # 处理视频的时长

dataset = datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)  # 读取对应数据集的类方法

print('All Videos: ', dataset.numVideos)
data_file_path = "D:\Data_Set\pyVHR\MovingClass\Class2\Data"
label_file_path = "D:\Data_Set\pyVHR\MovingClass\Class2\Labels"

In [ ]:
index = 96
videoFileName = dataset.getVideoFilename(index)
file_name = videoFileName[-14:-12]+videoFileName[-11:-9]
data_path0 = data_file_path + '\\' + file_name
label_path0 = label_file_path + '\\' + file_name
label = np.array([int(videoFileName[-11:-9])])
print('Video processed name: ', videoFileName)
fps = 30

index_data_file = videoFileName[:-4] + "-1-patch_index.txt"
X_data_file = videoFileName[:-4] + "-1-X_Motion.txt"
Y_data_file = videoFileName[:-4] + "-1-Y_Motion.txt"
Z_data_file = videoFileName[:-4] + "-1-Z_Motion.txt"

inx_data = np.loadtxt(index_data_file)
x_data = np.loadtxt(X_data_file)
y_data = np.loadtxt(Y_data_file)
z_data = np.loadtxt(Z_data_file)

motion_sig = np.array([inx_data, x_data, y_data], np.int32).transpose((1, 2, 0))
speed_x, speed_y, acc_x, acc_y = speed_acc_sig(motion_sig, 30)  # 数组形式速度、加速度信号
speed_x_sum, speed_y_sum, acc_x_sum, acc_y_sum, motion_sum = sum_speed_acc_sig(speed_x, speed_y, acc_x, acc_y, motion_sig)  # patches速度加速度求和
speed_x_sum = np.squeeze(speed_x_sum) / 42
speed_y_sum = np.squeeze(speed_y_sum) / 42
speed_z_sum = np.zeros_like(z_data) / 42
speed_z_sum[1:] = np.diff(z_data) * 30

In [ ]:
xx = speed_x_sum.copy()
yy = speed_y_sum.copy()
zz = speed_z_sum.copy()
fx, tx, energy_x = scipy.signal.stft(np.squeeze(xx), fs=30, nperseg=200, noverlap=196, nfft=1024)
fy, ty, energy_y = scipy.signal.stft(np.squeeze(yy), fs=30, nperseg=200, noverlap=196, nfft=1024)
fz, tz, energy_z = scipy.signal.stft(np.squeeze(zz), fs=30, nperseg=200, noverlap=196, nfft=1024)
en_x = np.abs(energy_x) / np.max(np.max(np.abs(energy_x)))
en_y = np.abs(energy_y) / np.max(np.max(np.abs(energy_y)))
en_z = np.abs(energy_z) / np.max(np.max(np.abs(energy_z)))
energy_xyz = en_x + en_y + en_z
f_length = int(4 / fx[1])+1

plt.figure(figsize=(8, 7))
plt.rcParams.update({'font.size': 18, 'font.family': 'Times New Roman'})
plt.pcolormesh(tx, fx[:f_length], np.abs(energy_xyz[:f_length, :]), cmap='hsv')
plt.ylabel('Frequency / Hz')
plt.xlabel('Time / sec')
plt.ylim((0, 4))
plt.tight_layout()
plt.show()

In [ ]:
lab = 1
label = np.array([lab])
label

In [ ]:
# 运动信号
speed = np.array([[speed_x_sum, speed_y_sum, speed_z_sum]]).transpose((2, 0, 1))
windowed_speed, _ = extraction.sig_windowing(speed, w_size, stride=stride, fps=fps)     # 获取RGB原始窗信号
win = 1
for win_speed in windowed_speed:
    spx = win_speed[0][0]
    spy = win_speed[0][1]
    spz = win_speed[0][2]
    fx, tx, spec_x = scipy.signal.stft(spx, fs=fps, nperseg=200, noverlap=196, nfft=1024)
    fy, ty, spec_y = scipy.signal.stft(spy, fs=fps, nperseg=200, noverlap=196, nfft=1024)
    fz, tz, spec_z = scipy.signal.stft(spz, fs=fps, nperseg=200, noverlap=196, nfft=1024)
    spec_x = np.abs(spec_x) / np.max(np.max(np.abs(spec_x)))
    spec_y = np.abs(spec_y) / np.max(np.max(np.abs(spec_y)))
    spec_z = np.abs(spec_z) / np.max(np.max(np.abs(spec_z)))
    spec_xyz = spec_x + spec_y + spec_z
    data_path1 = data_path0 + '_' + str(win) + '.txt'
    label_path1 = label_path0 + '_' + str(win) + '.txt'
    np.savetxt(data_path1, spec_xyz)
    np.savetxt(label_path1, label)
    print(win)
    win += 1